In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import warnings
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import platform

warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cpu')

In [2]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE': 1024,
    'SEED':41
}

PATH = os.getcwd() + '/data/'
if platform.system() == 'Darwin':
    LOADPATH = '/Users/a1r/Desktop/DL/timeseries_new_data/'
else:
    LOADPATH = '/home/a1r/바탕화면/DL/timeseries_new_data/'

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [6]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

#### CNN-LSTM

In [7]:
class Conv1d_LSTM(nn.Module):
    def __init__(self, in_channel=9, hidden_size = 128, out_channel=CFG['PREDICT_SIZE']):
        super(Conv1d_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.conv1d_1 = nn.Conv1d(in_channels=in_channel,
                                out_channels=16,
                                kernel_size=3,
                                stride=1,
                                padding=1)
        
        self.conv1d_2 = nn.Conv1d(in_channels=16,
                                out_channels=32,
                                kernel_size=3,
                                stride=1,
                                padding=1)
        
        self.lstm = nn.LSTM(input_size = 32,
                            hidden_size=hidden_size,
                            num_layers=1,
                            bias=True,
                            bidirectional=False,
                            batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, out_channel),
        )

        self.actv = nn.ReLU()
    
    def init_hidden(self, batch_size, device):
        return (
            torch.zeros(1, batch_size, self.hidden_size, device = device),  # h0
            torch.zeros(1, batch_size, self.hidden_size, device = device)   # c0
        )

    def forward(self, x):
	# Raw x shape : (B, TRAIN_WINDOW_SIZE, in_channel) => (B, 90, 9)
        
        # Shape : (B, F: in_channel, S: TRAIN_WINDOW_SIZE) => (B, 9, 90)
        x = x.transpose(1, 2)
        # Shape with Conv1d_1 : (B, F, S) == (B, C, S) || C = channel => (B, 16, 90)
        x = self.conv1d_1(x)
        ## Shape with Conv1d_2 : (B, C, S) => (B, 32, 90)
        x = self.conv1d_2(x)
        ## Reshape : (B, S, C) == (B, S, F) => (B, 90, 32)
        x = x.transpose(1, 2)
        
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # LSTM Layer
        lstm_out, hidden = self.lstm(x, hidden)

        # Only use the last output(-1) sequence
        last_output = lstm_out[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))
        x = output.squeeze(1)

        return x

#### GRU

In [8]:
class GRU(nn.Module):
    def __init__(self, in_channel=9, hidden_size = 512, out_channel=CFG['PREDICT_SIZE']):
            super(GRU, self).__init__()
            self.hidden_size = hidden_size
            self.input_size = in_channel

            self.gru = nn.GRU(input_size = self.input_size,
                        hidden_size = self.hidden_size,
                        num_layers = 1,
                        bias=True,
                        bidirectional=False,
                        batch_first=True)
            
            self.fc = nn.Sequential(
                  nn.Linear(hidden_size, hidden_size//2),
                  nn.ReLU(),
                  nn.Dropout(),
                  nn.Linear(hidden_size//2, out_channel)
                  )
            self.actv = nn.ReLU()
    
    def forward(self, x):
        # x shape: (B: batch_size, TRAIN_WINDOW_SIZE: 90, 9)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)
        
        # gru layer
        gru_out, hidden = self.gru(x, hidden)
        
        # Only use the last output sequence
        last_output = gru_out[:, -1, :]
        
        # Fully connected layer
        output = self.actv(self.fc(last_output))
        
        return output.squeeze(1)
    
    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return Variable(torch.zeros(1, batch_size, self.hidden_size, device = device)) # h0

### Ensemble

In [9]:
class Ensemble(nn.Module):
    def __init__(self, model1, model2):
        super(Ensemble, self).__init__()
        self.model1 = model1
        self.model2 = model2
    
    def forward(self, x):
        out1 = self.model1(x)   # out1.shape: (Batch_size, 21)
        out2 = self.model2(x)   # out2.shape: (Batch_size, 21)

        out = out1 + out2
        x = torch.softmax(out, dim = 1) # x.shape: (Batch_size, 21)
        return x

### Call the Train Data

In [8]:
train_input = np.load(LOADPATH + 'train_input_minmax.npy')
train_target = np.load(LOADPATH + 'train_target_minmax.npy')

In [9]:
data_len = len(train_input)
train_dataset = CustomDataset(train_input[:-int(data_len*0.2)], train_target[:-int(data_len*0.2)])
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(train_input[-int(data_len*0.2):], train_target[-int(data_len*0.2):])
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### Train

In [10]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    train_loss = {}
    val_loss = {}
    
    # Epoch
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss_li = []
        train_mae = []
    
    # Iteration
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss_li.append(loss.item())
        
        val_loss_li = validation(model, val_loader, criterion, device)
        val_loss_mean = np.mean(val_loss_li)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss_li):.5f}] Val Loss : [{val_loss_mean:.5f}]')
        
        if best_loss > val_loss_mean:
            best_loss = val_loss_mean
            best_model = model
            print('Model Saved')

        train_loss[epoch] = train_loss_li
        val_loss[epoch] = val_loss_li

    return best_model, train_loss, val_loss

def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return val_loss

In [11]:
cnn_lstm = Conv1d_LSTM()
gru = GRU()
model = Ensemble(cnn_lstm, gru)
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['LEARNING_RATE'])
infer_model, train_loss, val_loss = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/4010 [00:00<?, ?it/s]

  0%|          | 0/1003 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.03465] Val Loss : [0.03383]
Model Saved


  0%|          | 0/4010 [00:00<?, ?it/s]

  0%|          | 0/1003 [00:00<?, ?it/s]

...

  0%|          | 0/1003 [00:00<?, ?it/s]

Epoch : [9] Train Loss : [0.03417] Val Loss : [0.03333]
Model Saved


  0%|          | 0/4010 [00:00<?, ?it/s]

  0%|          | 0/1003 [00:00<?, ?it/s]

Epoch : [10] Train Loss : [0.03416] Val Loss : [0.03340]


### Save Model

In [12]:
SAVEPATH = os.getcwd() + '/Ensemble_lstm_gru.pth'
torch.save(infer_model.state_dict(), SAVEPATH)

train_loss = np.array(train_loss)
val_loss = np.array(val_loss)

# loss값 저장
np.save(PATH + 'train_loss_gru', train_loss)
np.save(PATH + 'val_loss_gru', val_loss)

### Load Model

In [10]:
cnn_lstm = Conv1d_LSTM()
gru = GRU()
lstm_gru = Ensemble(cnn_lstm, gru)

In [12]:
if platform.system() == 'Darwin':
    lstm_gru.load_state_dict(torch.load(os.getcwd() + "/Ensemble_lstm_gru.pth", map_location=torch.device('cpu')))
    cnn_lstm.load_state_dict(torch.load(os.getcwd() + '/cnn_lstm_1e_5.pth', map_location=torch.device('cpu')))
    gru.load_state_dict(torch.load(os.getcwd() + '/GRU_1e_4.pth', map_location=torch.device('cpu')))

else:
    lstm_gru.load_state_dict(torch.load(os.getcwd() + "Ensemble_lstm_gru.pth"))
    cnn_lstm.load_state_dict(torch.load(os.getcwd() + '/cnn_lstm_1e_5.pth'))
    gru.load_state_dict(torch.load(os.getcwd() + '/GRU_1e_4.pth'))

### Predictions

In [15]:
test_input = np.load(LOADPATH + 'test_input_minmax.npy')
min_ = np.load(PATH + 'MIN.npy')
max_ = np.load(PATH + 'MAX.npy')

In [16]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    predictions = []
    model.to(device)
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [18]:
# ens = Ensemble(cnn_lstm, gru)
predictions = inference(lstm_gru, test_loader, device)

  0%|          | 0/16 [00:00<?, ?it/s]

In [19]:
def return_scaler(arr, min_ = min_, max_ = max_):
    denom = max_ - min_
    for i in tqdm(range(len(arr))):
        arr[i] = (arr[i]*denom[i]) + min_[i]
    
    return arr

prediction_unscale = np.around(return_scaler(predictions))
prediction_unscale

  0%|          | 0/15890 [00:00<?, ?it/s]

array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
submit = pd.read_csv(PATH + '/sample_submission.csv') # submit은 15690개 상품 / 예측은 15682개 상품
submit.iloc[:, 1:] = prediction_unscale
submit.head(3)

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [18]:
SAVE_SUBMIT = os.getcwd() + '/data/Ensemble_cnn_lstm_gru.csv'
submit.to_csv(SAVE_SUBMIT, index_label=False)

In [19]:
pd.read_csv(SAVE_SUBMIT).head(3)

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,1,1,1,1,0,0,1,1,...,1,0,0,1,1,1,1,1,0,0
1,1,0,1,1,1,0,0,0,0,1,...,0,0,0,0,1,1,1,0,0,0
2,2,2,2,2,2,2,1,1,2,2,...,2,1,1,2,2,2,2,2,1,1
